In [1]:
import json
import numpy as np
from PIL import Image

In [3]:
with open('./hy_info/task3/qas/infographicsVQA_train_v1.0.json', 'r') as file:
    data_train = json.load(file)

with open('./hy_info/task3/qas/infographicsVQA_val_v1.0_withQT.json', 'r') as file:
    data_valid = json.load(file)

In [7]:
# data_train
# data_valid

In [8]:
# with open('/home/ges/level3-cv-productserving-cv-10/data/infographics/qas/infographicsVQA_val_v1.0_withQT.json', 'r') as file:
#     data = json.load(file)
# data

In [12]:
def image_to_npy(data,file_name):    
    imdb_data = []

    i = 0

    for item in data['data']:
        
        image_path = f"./hy_info/task3/images/{item['image_local_name']}"  # 이미지 폴더 경로 수정
        
        # 이미지 크기 정보를 읽음
        with Image.open(image_path) as img:
            image_width, image_height = img.size
        
        
        # OCR 출력 파일 로드
        with open(f"./hy_info/task3/ocrs/{item['ocr_output_file']}", 'r') as ocr_file:
            ocr_data = json.load(ocr_file)
        ocr_tokens = []
        ocr_normalized_boxes = []
        
        # print(ocr_data.keys())
        if 'WORD' in ocr_data :
            for word_info in ocr_data['WORD']:
            # OCR 토큰 추가
                ocr_tokens.append(word_info['Text'])

            # BoundingBox 정보를 사용하여 정규화된 박스 생성
            # 'Left', 'Top'은 박스의 좌상단 좌표, 'Width', 'Height'는 박스의 너비와 높이입니다.
                bounding_box = word_info['Geometry']['BoundingBox']
                
                normalized_box = [
                    bounding_box['Left'],
                    bounding_box['Top'],
                    bounding_box['Left'] + bounding_box['Width'],
                    bounding_box['Top'] + bounding_box['Height']
                ]
                ocr_normalized_boxes.append(normalized_box)
        else :
            print(item)
            print(False)
            break
        
        # print(ocr_tokens)
        # print(ocr_normalized_boxes)
        # 이미지 사이즈, OCR 정보 등을 필요에 따라 처리
        # 예시 코드에서는 OCR 데이터에서 직접적인 정보를 추출하지 않으며, 실제 구현에 필요한 로직 추가가 필요합니다.

        data_item = {
            'question': item['question'],
            'image_id': item['image_local_name'].split('.')[0],  # 이미지 ID를 파일 이름에서 추출
            'image_classes': None,  # 필요한 경우 채워넣어야 함
            'extra_info': item.get('image_url'),  # 추가 정보로 이미지 URL 사용
            'image_width': image_width,  # 이미지 처리를 통해 추출
            'image_height': image_height,  # 이미지 처리를 통해 추출
            'question_tokens': None,  # NLP 라이브러리를 사용해 질문 토큰화
            'question_id': item['questionId'],
            'set_name': item['data_split'],
            'image_name': item['image_local_name'],
            'image_path': f"images/{item['image_local_name']}",  # 이미지 파일 경로
            'feature_path': None,  # 이미지 특성 처리 후의 경로, 필요시 설정
            # 'ocr_tokens': ocr_data.get('tokens', []),  # OCR 데이터 구조에 따라 조정 필요
            'ocr_tokens': ocr_tokens,  # OCR 데이터 구조에 따라 조정 필요
            'ocr_info': ocr_data,  # 전체 OCR 데이터
            'ocr_normalized_boxes': ocr_normalized_boxes,  # OCR 데이터에서 추출, 필요시 처리
            'obj_normalized_boxes': None,  # 객체 인식 결과, 필요시 처리
            'answers': item['answers'],
            'valid_answers': item['answers']  # 유효한 답변, 처리 방식에 따라 다를 수 있음
        }
        imdb_data.append(data_item)
        
        

    # print(imdb_data[0]['ocr_tokens'])
    # print(imdb_data[0]['question'])
    # print(imdb_data[0]['image_id'])

    # print(imdb_data[0]['question_id'])
    # print(imdb_data[0]['image_name'])
    # for item in imdb_data[0]['ocr_info']['WORD']:
    #     print(item)
    # NumPy 배열로 변환
    imdb_array = np.array(imdb_data, dtype=object)
    np.save(file_name, imdb_array)

In [13]:
image_to_npy(data_train,'infographics_imdb_train.npy')
image_to_npy(data_valid,'infographics_imdb_val.npy')

## Test .npy 만드는 법

In [14]:
with open('./hy_info/task3/qas/infographicsVQA_test_v1.0.json', 'r') as file:
    data_test = json.load(file)
data_test

{'dataset_name': 'InfographicsVQA',
 'dataset_version': '1.0',
 'dataset_split': 'test',
 'data': [{'questionId': 98304,
   'question': 'Which category of apps enjoy the least percentage of popularity?',
   'image_local_name': '37143.jpeg',
   'image_url': 'https://blog.hubspot.com/hs-fs/hub/53/file-250043455-jpg/Blog-Related_Images/KontestappInfographic.jpg?width=645&amp;name=KontestappInfographic.jpg',
   'ocr_output_file': '37143.json',
   'data_split': 'test'},
  {'questionId': 98309,
   'question': 'Which social media network is used by fans to organize contest, Facebook, Twitter, or Whatsapp?',
   'image_local_name': '37143.jpeg',
   'image_url': 'https://blog.hubspot.com/hs-fs/hub/53/file-250043455-jpg/Blog-Related_Images/KontestappInfographic.jpg?width=645&amp;name=KontestappInfographic.jpg',
   'ocr_output_file': '37143.json',
   'data_split': 'test'},
  {'questionId': 98311,
   'question': 'What forms is the second least popular category of prizes?',
   'image_local_name': '3

In [15]:
imdb_data = []

i = 0

for item in data_test['data']:
    
    image_path = f"./hy_info/task3/images/{item['image_local_name']}"  # 이미지 폴더 경로 수정
    
    # 이미지 크기 정보를 읽음
    with Image.open(image_path) as img:
        image_width, image_height = img.size
    
    
    # OCR 출력 파일 로드
    with open(f"./hy_info/task3/ocrs/{item['ocr_output_file']}", 'r') as ocr_file:
        ocr_data = json.load(ocr_file)
    ocr_tokens = []
    ocr_normalized_boxes = []
    
    # print(ocr_data.keys())
    if 'WORD' in ocr_data :
        for word_info in ocr_data['WORD']:
        # OCR 토큰 추가
            ocr_tokens.append(word_info['Text'])

        # BoundingBox 정보를 사용하여 정규화된 박스 생성
        # 'Left', 'Top'은 박스의 좌상단 좌표, 'Width', 'Height'는 박스의 너비와 높이입니다.
            bounding_box = word_info['Geometry']['BoundingBox']
            
            normalized_box = [
                bounding_box['Left'],
                bounding_box['Top'],
                bounding_box['Left'] + bounding_box['Width'],
                bounding_box['Top'] + bounding_box['Height']
            ]
            ocr_normalized_boxes.append(normalized_box)
    else :
        print(False)
    
    # print(ocr_tokens)
    # print(ocr_normalized_boxes)
    # 이미지 사이즈, OCR 정보 등을 필요에 따라 처리
    # 예시 코드에서는 OCR 데이터에서 직접적인 정보를 추출하지 않으며, 실제 구현에 필요한 로직 추가가 필요합니다.

    data_item = {
        'question': item['question'],
        'image_id': item['image_local_name'].split('.')[0],  # 이미지 ID를 파일 이름에서 추출
        'image_classes': None,  # 필요한 경우 채워넣어야 함
        'extra_info': item.get('image_url'),  # 추가 정보로 이미지 URL 사용
        'image_width': image_width,  # 이미지 처리를 통해 추출
        'image_height': image_height,  # 이미지 처리를 통해 추출
        'question_tokens': None,  # NLP 라이브러리를 사용해 질문 토큰화
        'question_id': item['questionId'],
        'set_name': item['data_split'],
        'image_name': item['image_local_name'],
        'image_path': f"images/{item['image_local_name']}",  # 이미지 파일 경로
        'feature_path': None,  # 이미지 특성 처리 후의 경로, 필요시 설정
        # 'ocr_tokens': ocr_data.get('tokens', []),  # OCR 데이터 구조에 따라 조정 필요
        'ocr_tokens': ocr_tokens,  # OCR 데이터 구조에 따라 조정 필요
        'ocr_info': ocr_data,  # 전체 OCR 데이터
        'ocr_normalized_boxes': ocr_normalized_boxes,  # OCR 데이터에서 추출, 필요시 처리
        'obj_normalized_boxes': None,  # 객체 인식 결과, 필요시 처리
        # test 만들때는 정답 빼줘야함
        # 'answers': item['answers'],
        # 'valid_answers': item['answers']  # 유효한 답변, 처리 방식에 따라 다를 수 있음  
    }
    imdb_data.append(data_item)
    
    

# print(imdb_data[0]['ocr_tokens'])
# print(imdb_data[0]['question'])
# print(imdb_data[0]['image_id'])

# print(imdb_data[0]['question_id'])
# print(imdb_data[0]['image_name'])
# for item in imdb_data[0]['ocr_info']['WORD']:
#     print(item)
# NumPy 배열로 변환
imdb_array = np.array(imdb_data, dtype=object)

In [16]:
np.save('infographics_imdb_test.npy', imdb_array)